#OSTA NLP

In [10]:
# Install required packages
!pip install nltk beautifulsoup4 wordcloud tqdm matplotlib seaborn pandas requests

# Import necessary libraries
import nltk
import pandas as pd
import numpy as np
import requests
import logging
import concurrent.futures
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
from datetime import datetime, timedelta
from tqdm.notebook import tqdm  # Use notebook version for Colab
import os
import pickle
from typing import Dict, List, Tuple, Optional
from IPython.display import display, HTML, clear_output

# Create directories for storing cache and results locally
BASE_DIR = './StockSentiment'
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(f"{BASE_DIR}/cache", exist_ok=True)
os.makedirs(f"{BASE_DIR}/results", exist_ok=True)
os.makedirs(f"{BASE_DIR}/plots", exist_ok=True)
print(f"Data will be stored in {BASE_DIR}")

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Download required NLTK data
nltk.download('vader_lexicon', quiet=True)

class StockSentimentAnalyzer:
    """Class for analyzing sentiment of stock news headlines"""

    def __init__(self, tickers: List[str], use_cached: bool = True, cache_dir: str = f"{BASE_DIR}/cache"):
        """
        Initialize the StockSentimentAnalyzer

        Args:
            tickers: List of stock ticker symbols
            use_cached: Whether to use cached data if available
            cache_dir: Directory to store cached data
        """
        self.tickers = tickers
        self.finviz_url = 'https://finviz.com/quote.ashx?t='
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.use_cached = use_cached
        self.cache_dir = cache_dir

        # Create cache directory if it doesn't exist
        os.makedirs(cache_dir, exist_ok=True)

        self.sia = SentimentIntensityAnalyzer()
        self.news_df = None
        self.daily_sentiment_df = None

    def _fetch_news_for_ticker(self, ticker: str) -> Optional[BeautifulSoup]:
        """
        Fetch news for a specific ticker

        Args:
            ticker: Stock ticker symbol

        Returns:
            BeautifulSoup object containing the news table
        """
        cache_file = os.path.join(self.cache_dir, f"{ticker}_news.pkl")
        url = self.finviz_url + ticker

        logger.info(f"Fetching data for {ticker} from {url}")

        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            logger.info(f"Response status: {response.status_code}")

            # Check if we got blocked
            if "Access denied" in response.text:
                logger.error("Finviz blocked the request (access denied)")
                return None

            soup = BeautifulSoup(response.text, 'html.parser')

            # Debug: Save the HTML for inspection
            debug_file = os.path.join(self.cache_dir, f"{ticker}_debug.html")
            with open(debug_file, 'w', encoding='utf-8') as f:
                f.write(response.text)
            logger.info(f"Saved raw HTML to {debug_file}")

            # Look for the news table - try different selectors
            news_table = soup.find(id='news-table')
            if not news_table:
                news_table = soup.find('table', {'class': 'news-table'})
                logger.warning("Tried alternative selector for news table")

            if news_table:
                logger.info(f"Found news table with {len(news_table.findAll('tr'))} rows")
                # Cache the results
                with open(cache_file, 'wb') as f:
                    pickle.dump({
                        'timestamp': datetime.now(),
                        'news_table': news_table
                    }, f)
                return news_table
            else:
                logger.error("Could not find news table in HTML")
                return None

        except Exception as e:
            logger.error(f"Error fetching data for {ticker}: {str(e)}")
            return None

    def fetch_all_news(self) -> Dict[str, BeautifulSoup]:
        """
        Fetch news for all tickers in parallel

        Returns:
            Dictionary mapping tickers to news tables
        """
        news_tables = {}
        logger.info(f"Fetching news for {len(self.tickers)} tickers...")

        with concurrent.futures.ThreadPoolExecutor(max_workers=min(5, len(self.tickers))) as executor:
            # Map tickers to futures
            future_to_ticker = {executor.submit(self._fetch_news_for_ticker, ticker): ticker for ticker in self.tickers}

            # Process results as they complete
            for future in tqdm(concurrent.futures.as_completed(future_to_ticker), total=len(self.tickers), desc="Fetching news"):
                ticker = future_to_ticker[future]
                try:
                    news_table = future.result()
                    if news_table:
                        news_tables[ticker] = news_table
                except Exception as e:
                    logger.error(f"Error processing {ticker}: {e}")

        return news_tables

    def process_news_data(self, news_tables: Dict[str, BeautifulSoup]) -> pd.DataFrame:
        """
        Process news tables into a DataFrame

        Args:
            news_tables: Dictionary mapping tickers to news tables

        Returns:
            DataFrame containing processed news data
        """
        columns = ['ticker', 'date', 'time', 'title']
        news_list = []

        for ticker, news_table in news_tables.items():
            if news_table is None:
                continue

            for row in news_table.findAll('tr'):
                title_element = row.a
                if not title_element:
                    continue

                title = title_element.text

                date_element = row.td
                if not date_element or not date_element.text:
                    continue

                date_data = date_element.text.strip().split()

                if len(date_data) == 1:
                    # If only time is available, use today's date
                    time = date_data[0]
                    date = datetime.now().strftime('%b-%d-%y')
                else:
                    date = date_data[0]
                    time = date_data[1]

                news_list.append([ticker, date, time, title])

        # Create DataFrame
        news_df = pd.DataFrame(news_list, columns=columns)

        if news_df.empty:
            logger.warning("No news data found for any ticker")
            return pd.DataFrame(columns=columns + ['date_only'])

        try:
            # First try parsing with current year
            current_year = datetime.now().year
            news_df['date'] = pd.to_datetime(
                news_df['date'] + '-' + str(current_year) + ' ' + news_df['time'],
                format='%b-%d-%Y %I:%M%p',
                errors='coerce'
            )

            # For any that failed, try without year (using default year)
            mask = news_df['date'].isna()
            if mask.any():
                news_df.loc[mask, 'date'] = pd.to_datetime(
                    news_df.loc[mask, 'date'] + ' ' + news_df.loc[mask, 'time'],
                    format='%b-%d-%y %I:%M%p',
                    errors='coerce'
                )
        except Exception as e:
            logger.error(f"Error parsing dates: {e}")
            return pd.DataFrame()

        # Extract just the date component
        news_df['date_only'] = news_df['date'].dt.date

        return news_df

    def calculate_sentiment(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate sentiment scores for news headlines

        Args:
            df: DataFrame containing news data

        Returns:
            DataFrame with added sentiment scores
        """
        if df.empty:
            logger.warning("No data to calculate sentiment for")
            return df

        # Calculate sentiment metrics
        df['compound_score'] = df['title'].apply(lambda x: self.sia.polarity_scores(x)['compound'])
        df['positive_score'] = df['title'].apply(lambda x: self.sia.polarity_scores(x)['pos'])
        df['negative_score'] = df['title'].apply(lambda x: self.sia.polarity_scores(x)['neg'])
        df['neutral_score'] = df['title'].apply(lambda x: self.sia.polarity_scores(x)['neu'])

        return df

    def aggregate_daily_sentiment(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Aggregate sentiment scores by ticker and date

        Args:
            df: DataFrame containing news data with sentiment scores

        Returns:
            DataFrame with aggregated daily sentiment
        """
        if df.empty or 'compound_score' not in df.columns:
            logger.warning("No sentiment data to aggregate")
            return pd.DataFrame()

        # Group by ticker and date
        daily_df = df.groupby(['ticker', 'date_only']).agg({
            'compound_score': ['mean', 'count', 'std'],
            'positive_score': 'mean',
            'negative_score': 'mean',
            'neutral_score': 'mean',
            'title': lambda x: ' '.join(x)  # Combine titles for word cloud
        }).reset_index()

        # Flatten multi-level columns
        daily_df.columns = ['ticker', 'date', 'avg_compound', 'news_count', 'std_compound',
                           'avg_positive', 'avg_negative', 'avg_neutral', 'all_titles']

        # Calculate normalized scores (handling division by zero)
        means = daily_df.groupby('ticker')['avg_compound'].transform('mean')
        stds = daily_df.groupby('ticker')['avg_compound'].transform('std').replace(0, 1)
        daily_df['normalized_score'] = (daily_df['avg_compound'] - means) / stds

        # Calculate sentiment momentum (change from previous day)
        daily_df['sentiment_change'] = daily_df.groupby('ticker')['avg_compound'].diff()

        # Sort by ticker and date
        daily_df = daily_df.sort_values(['ticker', 'date'])

        return daily_df

    def analyze(self, start_date: Optional[datetime] = None, end_date: Optional[datetime] = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Perform the complete sentiment analysis process

        Args:
            start_date: Optional start date for filtering
            end_date: Optional end date for filtering

        Returns:
            Tuple of (news DataFrame, daily sentiment DataFrame)
        """
        # Fetch news
        news_tables = self.fetch_all_news()

        # Process news data
        self.news_df = self.process_news_data(news_tables)

        if self.news_df.empty:
            logger.warning("No news data found, returning empty DataFrames")
            return pd.DataFrame(), pd.DataFrame()

        # Filter by date range if specified
        if start_date and end_date:
            self.news_df = self.news_df[
                (self.news_df['date_only'] >= start_date.date()) &
                (self.news_df['date_only'] <= end_date.date())
            ]

        # Calculate sentiment scores
        self.news_df = self.calculate_sentiment(self.news_df)

        # Aggregate daily sentiment
        self.daily_sentiment_df = self.aggregate_daily_sentiment(self.news_df)

        return self.news_df, self.daily_sentiment_df

    def plot_sentiment_trends(self, output_dir: str = f"{BASE_DIR}/plots", display_inline: bool = True):
        """
        Generate plots for sentiment analysis results

        Args:
            output_dir: Directory to save plots
            display_inline: Whether to display plots inline in the notebook
        """
        if self.daily_sentiment_df is None or self.daily_sentiment_df.empty:
            raise ValueError("No data available. Run analyze() first.")

        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Set plot style
        sns.set_style('whitegrid')
        sns.set_context("talk")

        for ticker in self.tickers:
            ticker_df = self.daily_sentiment_df[self.daily_sentiment_df['ticker'] == ticker]

            if ticker_df.empty:
                logger.warning(f"No data available for {ticker}")
                continue

            # Plot normalized sentiment scores
            plt.figure(figsize=(12, 6))
            ax = sns.lineplot(x='date', y='normalized_score', data=ticker_df, marker='o')
            plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
            plt.title(f'Normalized Sentiment Analysis for {ticker}')
            plt.xlabel('Date')
            plt.ylabel('Normalized Sentiment Score')
            plt.xticks(rotation=45)
            plt.tight_layout()

            if display_inline:
                plt.show()

            plt.savefig(os.path.join(output_dir, f'{ticker}_sentiment.png'), dpi=300)
            if not display_inline:
                plt.close()

            # Plot news volume
            plt.figure(figsize=(12, 6))
            ax = sns.barplot(x='date', y='news_count', data=ticker_df)
            plt.title(f'News Volume for {ticker}')
            plt.xlabel('Date')
            plt.ylabel('Number of News Articles')
            plt.xticks(rotation=45)
            plt.tight_layout()

            if display_inline:
                plt.show()

            plt.savefig(os.path.join(output_dir, f'{ticker}_volume.png'), dpi=300)
            if not display_inline:
                plt.close()

            # Create word cloud from all titles
            if not ticker_df['all_titles'].empty and isinstance(ticker_df['all_titles'].iloc[0], str) and ticker_df['all_titles'].iloc[0].strip():
                plt.figure(figsize=(12, 8))
                wordcloud = WordCloud(width=800, height=400, background_color='white',
                                     colormap='viridis', max_words=100).generate(ticker_df['all_titles'].iloc[0])
                plt.imshow(wordcloud, interpolation='bilinear')
                plt.axis('off')
                plt.title(f'Word Cloud for {ticker} News')
                plt.tight_layout()

                if display_inline:
                    plt.show()

                plt.savefig(os.path.join(output_dir, f'{ticker}_wordcloud.png'), dpi=300)
                if not display_inline:
                    plt.close()

    def plot_comparative_analysis(self, output_dir: str = f"{BASE_DIR}/plots", display_inline: bool = True):
        """
        Generate comparative analysis plots across tickers

        Args:
            output_dir: Directory to save plots
            display_inline: Whether to display plots inline in the notebook
        """
        if self.daily_sentiment_df is None or self.daily_sentiment_df.empty:
            raise ValueError("No data available. Run analyze() first.")

        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Filter to only recent dates that all tickers have in common
        recent_dates = self.daily_sentiment_df['date'].unique()
        if len(recent_dates) > 0:
            recent_dates = sorted(recent_dates)[-min(10, len(recent_dates)):]  # Last 10 dates or fewer
            comp_df = self.daily_sentiment_df[self.daily_sentiment_df['date'].isin(recent_dates)]

            if not comp_df.empty:
                # Comparative sentiment plot
                plt.figure(figsize=(14, 8))
                sns.lineplot(data=comp_df, x='date', y='avg_compound', hue='ticker', marker='o')
                plt.title('Comparative Sentiment Analysis')
                plt.xlabel('Date')
                plt.ylabel('Average Compound Sentiment')
                plt.legend(title='Ticker')
                plt.xticks(rotation=45)
                plt.tight_layout()

                if display_inline:
                    plt.show()

                plt.savefig(os.path.join(output_dir, 'comparative_sentiment.png'), dpi=300)
                if not display_inline:
                    plt.close()

                # News volume comparison
                plt.figure(figsize=(14, 8))
                sns.barplot(data=comp_df, x='ticker', y='news_count', hue='date')
                plt.title('News Volume by Ticker')
                plt.xlabel('Ticker')
                plt.ylabel('Number of News Articles')
                plt.legend(title='Date', loc='upper right')
                plt.tight_layout()

                if display_inline:
                    plt.show()

                plt.savefig(os.path.join(output_dir, 'comparative_volume.png'), dpi=300)
                if not display_inline:
                    plt.close()

                # Heatmap of sentiment
                if len(comp_df['ticker'].unique()) > 1 and len(comp_df['date'].unique()) > 1:
                    pivot_df = comp_df.pivot(index='ticker', columns='date', values='avg_compound')
                    plt.figure(figsize=(14, 8))
                    sns.heatmap(pivot_df, annot=True, cmap='RdYlGn', center=0)
                    plt.title('Sentiment Heatmap by Ticker and Date')
                    plt.tight_layout()

                    if display_inline:
                        plt.show()

                    plt.savefig(os.path.join(output_dir, 'sentiment_heatmap.png'), dpi=300)
                    if not display_inline:
                        plt.close()

    def export_results(self, output_dir: str = f"{BASE_DIR}/results"):
        """
        Export analysis results to CSV files

        Args:
            output_dir: Directory to save results
        """
        if self.news_df is None or self.daily_sentiment_df is None:
            raise ValueError("No data available. Run analyze() first.")

        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Export the data
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.news_df.to_csv(os.path.join(output_dir, f'news_data_{timestamp}.csv'), index=False)
        self.daily_sentiment_df.to_csv(os.path.join(output_dir, f'daily_sentiment_{timestamp}.csv'), index=False)

        logger.info(f"Results exported to {output_dir}")

    def display_summary_tables(self):
        """Display summary tables of sentiment analysis in the notebook"""
        if self.news_df is None or self.daily_sentiment_df is None or self.daily_sentiment_df.empty:
            print("No data available. Run analyze() first.")
            return

        # Display recent news with sentiment
        print("\n=== Recent News with Sentiment Scores ===")
        recent_news = self.news_df.sort_values('date', ascending=False).head(10)[
            ['ticker', 'date', 'title', 'compound_score']
        ].rename(columns={'compound_score': 'sentiment'})

        # Apply color formatting
        def sentiment_color(val):
            if val > 0.2:
                return 'background-color: lightgreen'
            elif val < -0.2:
                return 'background-color: lightcoral'
            else:
                return 'background-color: lightyellow'

        styled_news = recent_news.style.applymap(sentiment_color, subset=['sentiment'])
        display(styled_news)

        # Ticker sentiment summary
        print("\n=== Sentiment Statistics by Ticker ===")
        ticker_stats = self.daily_sentiment_df.groupby('ticker').agg({
            'avg_compound': ['mean', 'min', 'max', 'std'],
            'news_count': 'sum'
        })

        # Flatten multi-level columns
        ticker_stats.columns = ['avg_sentiment', 'min_sentiment', 'max_sentiment', 'std_sentiment', 'total_news']
        ticker_stats = ticker_stats.reset_index()

        # Apply color formatting
        def avg_sentiment_color(val):
            if val > 0.1:
                return 'background-color: lightgreen'
            elif val < -0.1:
                return 'background-color: lightcoral'
            else:
                return 'background-color: lightyellow'

        styled_stats = ticker_stats.style.applymap(avg_sentiment_color, subset=['avg_sentiment'])
        display(styled_stats)

# Interactive Colab Interface

# Create interactive widgets for user input
from ipywidgets import widgets
from IPython.display import display

def run_analysis():
    # Clear previous outputs
    clear_output(wait=True)

    # Get values from widgets
    tickers = [t.strip() for t in ticker_input.value.split(',')]
    use_cache = cache_checkbox.value

    if start_date_input.value and end_date_input.value:
        start_date = datetime.strptime(start_date_input.value, '%Y-%m-%d')
        end_date = datetime.strptime(end_date_input.value, '%Y-%m-%d')
    else:
        start_date = None
        end_date = None

    # Create and run analyzer
    analyzer = StockSentimentAnalyzer(tickers, use_cached=use_cache)

    print(f"Analyzing sentiment for: {', '.join(tickers)}")
    if start_date and end_date:
        print(f"Date range: {start_date.date()} to {end_date.date()}")

    # Run analysis
    news_df, daily_df = analyzer.analyze(start_date, end_date)

    if news_df.empty:
        print("No news data found for the specified tickers.")
        return

    # Display summary
    print("\n=== Analysis Summary ===")
    print(f"Tickers analyzed: {', '.join(tickers)}")
    print(f"Total news articles: {len(news_df)}")
    print(f"Date range: {news_df['date_only'].min()} to {news_df['date_only'].max()}")

    # Display detailed summary tables
    analyzer.display_summary_tables()

    # Generate plots
    print("\nGenerating visualizations...")
    analyzer.plot_sentiment_trends(display_inline=True)
    analyzer.plot_comparative_analysis(display_inline=True)

    # Export results
    if export_checkbox.value:
        analyzer.export_results()
        print(f"\nResults exported to: {BASE_DIR}/results")

    print("\nAnalysis complete!")

# Create widgets
ticker_input = widgets.Text(
    value='AAPL,GOOG,MSFT,AMZN,TSLA',
    description='Tickers:',
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

start_date_input = widgets.Text(
    value=(datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d'),
    description='Start Date (YYYY-MM-DD):',
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

end_date_input = widgets.Text(
    value=datetime.now().strftime('%Y-%m-%d'),
    description='End Date (YYYY-MM-DD):',
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

cache_checkbox = widgets.Checkbox(
    value=True,
    description='Use cached data (if available)',
    style={'description_width': 'initial'}
)

export_checkbox = widgets.Checkbox(
    value=True,
    description='Export results to local directory',
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description='Run Analysis',
    button_style='success',
    tooltip='Click to start analysis'
)

run_button.on_click(lambda b: run_analysis())

# Display the interface
print("# Stock Sentiment Analysis Tool")
print("Enter stock tickers separated by commas and optionally specify a date range.")
display(ticker_input)
display(start_date_input)
display(end_date_input)
display(cache_checkbox)
display(export_checkbox)
display(run_button)

# Example usage (uncomment to run without the UI)
# Initialize analyzer
analyzer = StockSentimentAnalyzer(['AAPL', 'MSFT', 'GOOGL'])

# Run analysis for the last 30 days
end_date = datetime.now()
start_date = end_date - timedelta(days=30)
news_df, daily_df = analyzer.analyze(start_date, end_date)

# Display summary tables
analyzer.display_summary_tables()

# Generate plots
analyzer.plot_sentiment_trends()
analyzer.plot_comparative_analysis()

# Export results
analyzer.export_results()

Analyzing sentiment for: AAPL, GOOG, MSFT, AMZN, TSLA
Date range: 2025-03-06 to 2025-04-05


Fetching news:   0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-10-56ecac58c2b1>:111: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  logger.info(f"Found news table with {len(news_table.findAll('tr'))} rows")
ERROR:__main__:Error fetching data for AAPL: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Error fetching data for TSLA: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Error fetching data for GOOG: maximum recursion depth exceeded
ERROR:__main__:Error fetching data for MSFT: maximum recursion depth exceeded
ERROR:__main__:Error fetching data for AMZN: maximum recursion depth exceeded


No news data found for the specified tickers.
